In [ ]:
!pip install openai  # OpenAI API for prompting GPT-4o

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm

# Reading Excel Sheet

In [ ]:
path = "Rerun_input.xlsx"
workbook = openpyxl.load_workbook(path)
sheet = workbook.active  # There is only one sheet, so no ambiguity here.

Get all the responses and scores

In [ ]:
INPUT_COLUMN = 31 # Specify the column from which input is read (A->1, B->2, etc.)
row_count = 215 # sheet.max_row  # Get number of rows. If number of rows to read is already known (or sheet does not terminate where data terminates), place number here (+1) instead.
inputs = [sheet.cell(row=i, column=INPUT_COLUMN).value  for i in range(2, row_count + 1)]  # Column number is set here. YOUR INPUT COLUMN NUMBER GOES HERE.
inputs = [x for x in inputs if x is not None]

# OpenAI and Prompt Setup

In [ ]:
# Insert your API Key for OpenAI here.

Scoring Prompt Input

In [ ]:
SCORING_PROMPT_START = """
Please score the following tutor response to a tutor training scenario involving a middle school student facing a microaggression as follows:
-if the tutor’s response acknowledges a student facing a microaggression and provides support to the student, by giving the student space to talk, or facilitating an apology, score with a 1. Sample responses scoring a 1 are "Nori, I heard what your teacher said about the exit ticket. It's okay to find problems challenging. Let's work on it together during our session. Your efforts are important, and I'm here to support you." and "Nori, I noticed what your teacher said to you could have been upsetting and I am here if you would like to talk about it." Sample response scoring a 1 include: "Nori, I noticed what your teacher said to you could have been upsetting and I am here if you would like to talk about it." and "Nori, I heard what your teacher said about the exit ticket. It's okay to find problems challenging. Let's work on it together during our session. Your efforts are important, and I'm here to support you."
-if the tutor’s response does not acknowledge the microaggression or does not provide any support to the student, score with a 0. Sample responses scoring a 0 are "Nothing is too difficult if you decide you want to do it Nori. You are capable." and "Do want to talk about what the teacher said to you?"


Response Start ---
"""


SCORING_FORMAT_PROMPT = """
--- Response End. Given the earlier response, please return a JSON string following the format, {\"Rationale\": \"your reasoning here\", \"Score\":0/1}.
"""


Feedback Elaboration Prompt

In [ ]:
# Second prompt: Pass feedback and the original input to the rewriter. Can omit the original input later to save on token cost.

FEEDBACK_PROMPT_START = """
Please write the following third-person feedback about a tutor into the second person. In doing so, please address the tutor directly and do not exceed 100 words.
Please start with positive feedback, if any. Subsequently, please provide any critiques, if applicable, in a constructive tone.

Third-person Feedback Start ---
"""

# Design choice: Passing original input to potentially provide more concrete second-person feedback (original feedback could be high-level). Can omit to reduce cost
ORIGINAL_RESPONSE_PROMPT = """
--- Third-person Feedback End. For further context, the original feedback was provided for the following response:

Response Start ---
"""

FEEDBACK_FORMAT_PROMPT_WITH_RESPONSE = """
--- Response End. Please return your refined feedback, based on the original response and the third-person feedback, in a JSON string following the format, {\"Feedback\": \"your response here\"}.
"""

FEEDBACK_FORMAT_PROMPT_WITHOUT_RESPONSE = """
--- Third-person Feedback End. Please return your refined feedback, based on the third-person feedback, in a JSON string following the format, {\"Feedback\": \"your response here\"}.
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## OpenAI API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 300
TEMPERATURE = 0
RUN_UP_TO = -1  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
TWO_STAGE = False  # Specifies whether to refine the feedback provided by the scoring prompt
TWO_STAGE_INCLUDE_RESPONSE = False # Specifies whether the second-stage prompt uses the original response.
SCORE_COLUMN = 32  # Change column numbers here to  modify where output is written
RATIONALE_COLUMN= 33
REFINED_RATIONALE_COLUMN= 34


MODEL = "gpt-4o"

# Add titling to the new columns
output_score_title = sheet.cell(row=1, column=SCORE_COLUMN)  # Output Score Column goes here. CHANGE COLUMN NUMBER to set score output location
output_score_title.value = "OpenAI Score"  # OPTIONAL: Sets title cell in row 1. Comment out to disable this

rationale_title = sheet.cell(row=1, column=RATIONALE_COLUMN)
rationale_title.value = "OpenAI Rationale"

if TWO_STAGE:  # Creates a new column for the refined rationale
  refined_rationale_title = sheet.cell(row=1, column=REFINED_RATIONALE_COLUMN)
  if TWO_STAGE_INCLUDE_RESPONSE:  # If the second-stage prompt also uses the original response, use a different column title
    refined_rationale_title.value = "Refined Feedback (w/ Response in prompt)"
  else:
    refined_rationale_title.value = "Refined Feedback (w/o response in prompt)"


if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  overall_history = [{"role": "system", "content": SCORING_PROMPT_START}, {"role": "user", "content": inpt}, {"role": "system", "content": SCORING_FORMAT_PROMPT}]
  openai_out = client.chat.completions.create(model=MODEL, messages=overall_history, max_tokens=MAX_TOKENS, temperature = TEMPERATURE)
  role, content = extract_response(openai_out)
  # We now need to parse the JSON into rational and score
  score_cell = sheet.cell(row=index+2, column=SCORE_COLUMN) # Score cell
  rationale_cell = sheet.cell(row=index+2, column=RATIONALE_COLUMN)  # Rationale cell
  refined_feedback_cell = sheet.cell(row=index+2, column=REFINED_RATIONALE_COLUMN)  # Refined feedback cell
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale

    score_cell.value = score  # Now write both into the Excel sheet
    rationale_cell.value = rationale

    if TWO_STAGE:  # If we are to refine the feedback
      if TWO_STAGE_INCLUDE_RESPONSE: # If we also want to pass the response, then we need to use a longer prompt, specified next:
        refinement_history = [{"role": "system", "content": FEEDBACK_PROMPT_START}, {"role": "user", "content": rationale}, {"role": "system", "content": ORIGINAL_RESPONSE_PROMPT}, {"role": "user", "content": inpt}, {"role": "system", "content":FEEDBACK_FORMAT_PROMPT_WITH_RESPONSE}]
      else: # If not, then we just pass the shorter prompt without the response
        refinement_history = [{"role": "system", "content": FEEDBACK_PROMPT_START}, {"role": "user", "content": rationale}, {"role": "system", "content":FEEDBACK_FORMAT_PROMPT_WITHOUT_RESPONSE}]
      openai_ss_out = client.chat.completions.create(model=MODEL, messages=refinement_history, max_tokens=MAX_TOKENS, temperature = TEMPERATURE)  # TODO: Use different model,  temperature, max token for the second stage
      role_ss, content_ss = extract_response(openai_ss_out)  # ss: second stage
      try:
        refined_content_json = json.loads(content_ss)  #  Parse response into JSON
        refined_feedback = str(refined_content_json["Feedback"])
        refined_feedback_cell.value = refined_feedback  # Write the refined feedback into the excel sheet.
      except:
        refined_feedback_cell.value = "---"  # If OpenAI / parsing fails for whatever reason.
  except:
    score_cell.value = "---"
    rationale_cell.value = "---"  # Failsafe
    if TWO_STAGE:
      refined_feedback_cell.value = "---"

100%|██████████| 212/212 [04:56<00:00,  1.40s/it]


## Save Excel Sheet

In [ ]:
FILE_NAME = "OutputSheet_max"+str(MAX_TOKENS)+"tokens_temp"+str(TEMPERATURE)  # Change file name here.
if RUN_UP_TO >= 0:
  FILE_NAME = FILE_NAME+"_first"+str(RUN_UP_TO)+"resp"
if TWO_STAGE:
  FILE_NAME = FILE_NAME+"_twostage"
if TWO_STAGE_INCLUDE_RESPONSE:
  FILE_NAME = FILE_NAME+"_include_resp_in_prompt"

FILE_NAME = FILE_NAME + ".xlsx"

workbook.save(FILE_NAME)  # Save the new file with OpenAI output into the file system